In [2]:
def do_count( df, group_cols, agg_name, agg_type='uint32', show_max=False):
    print( "do_count to ", group_cols , '...' )
    gp = df[group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
#     if debug:
#         print(df[group_cols])
# #         print(df[group_cols][group_cols]) # same to the former
#         print(df[group_cols][group_cols].groupby(group_cols).size())
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def do_countuniq( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False):
    print( "Counting unqiue ", counted, " by ", group_cols , '...' )
    # print('the Id of train_df while function before merge: ',id(df)) # the same with train_df
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    # print('the Id of train_df while function after merge: ',id(df)) # id changes
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )
    
def do_cumcount( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False):
    print( "Cumulative count by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].cumcount() # 累加
    df[agg_name]=gp.values
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def do_mean( df, group_cols, counted, agg_name, agg_type='float32', show_max=False):
    print( "Calculating mean of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def do_var( df, group_cols, counted, agg_name, agg_type='float32', show_max=False):
    print( "Calculating variance of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].var().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

In [3]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import os
from pandas.tseries.offsets import DateOffset

# debug = True
debug=False

frm = 0
to = 184903891
nchunk=184903891 # entire dataset
val_size=62833089  # since 9th 0:00:00
if debug:
    print('*** Debug: this is a test run for debugging purposes ***')
    frm=0
    nchunk=100000
    val_size=10000
    to = 100000

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
       # 'os'            : 'uint16',
        'channel'       : 'uint16'
#         'is_attributed' : 'uint8', # 【consider bool?need test】
        }

print('loading train data...',frm,'to',to)
# usecols:Using this parameter results in much faster parsing time and lower memory usage.
train_df = pd.read_csv("../input/train.csv",skiprows=range(1,frm), nrows=to-frm, dtype=dtypes, usecols=['ip','app','device','channel'])

print('loading test data...')
if debug:
    test_df = pd.read_csv("../input/test_supplement.csv", nrows=100000, parse_dates=['click_time'], dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])
else:
    test_df = pd.read_csv("../input/test_supplement.csv",dtype=dtypes, usecols=['ip','app','device', 'channel'])

print('Combine test_df and train_df...')
len_train = len(train_df)
# Append rows of other to the end of this frame, returning a new object.
# 【consider using concat more effiencent?add two more useless columns?】
train_df=train_df.append(test_df) # Shouldn't process individually,because of lots of count,mean,var variables
# train_df['is_attributed'].fillna(-1,inplace=True)
# train_df['is_attributed'] = train_df['is_attributed'].astype('uint8',copy=False)
del test_df
gc.collect()

print('Extracting hour and day and add 8h...')
#     print(type(train_df['click_time']),type(pd.to_datetime(train_df.click_time))) # Series
#     print(train_df['click_time'],pd.to_datetime(train_df.click_time)) # dtype: datetime64[ns]
# http://pandas.pydata.org/pandas-docs/stable/timeseries.html
#     print(train_df.click_time.head(500)) # 500 data before 6th 16:00
# train_df.click_time = train_df.click_time + DateOffset(hours=8)
# #     print(train_df.click_time.head(500))
# train_df['hour'] = train_df.click_time.dt.hour.astype('uint8')
# train_df['day'] = train_df.click_time.dt.day.astype('uint8')
# gc.collect()

#=====================================================================================================
print('Combining new features...')

# train_df['click_time'] = (train_df['click_time'].astype(np.int64,copy=False) // 10 ** 9).astype(np.int32,copy=False)
# train_df['nextClick'] = (train_df.groupby(['ip', 'app', 'device', 'os']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)

# feature1：
# train_df['nextClick2'] = (train_df.groupby(['ip', 'app', 'device', 'os','channel']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)
# train_df['nextClick3'] = (train_df.groupby(['ip', 'channel', 'device', 'os']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)
# train_df['nextClick4'] = (train_df.groupby(['app', 'device', 'channel']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)
# train_df['nextClick5'] = (train_df.groupby(['ip', 'os', 'device']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)
# 好用！：['app', 'device', 'channel']，['ip', 'app', 'device', 'os','channel']，['ip', 'app', 'device', 'os']

# feature2：
# print(train_df.groupby(['ip','channel'])) # <pandas.core.groupby.DataFrameGroupBy object at 0x000002C9D7C522E8>
# print(train_df.groupby(['ip','channel']).click_time) # <pandas.core.groupby.SeriesGroupBy object at 0x000002C9D7479748>
# print(train_df.click_time)
# print(train_df.groupby(['ip','channel']).click_time.shift(+1))
# print(train_df.click_time - train_df.groupby(['ip','channel']).click_time.shift(+1))
# DataFrameGroupBy.shift:Shift index by desired number of periods with an optional time freq
# train_df['prevClick'] = (train_df.click_time - train_df.groupby(['ip','channel']).click_time.shift(+1)).astype(np.float32,copy=False)
# train_df['prevClick2'] = (train_df.click_time - train_df.groupby(['ip', 'app', 'device', 'os']).click_time.shift(+1)).astype(np.float32,copy=False)
# train_df['prevClick3'] = (train_df.click_time - train_df.groupby(['ip', 'app', 'device', 'os','channel']).click_time.shift(+1)).astype(np.float32,copy=False)
# train_df['prevClick4'] = (train_df.click_time - train_df.groupby(['app', 'device', 'channel']).click_time.shift(+1)).astype(np.float32,copy=False)
# train_df['prevClick5'] = (train_df.click_time - train_df.groupby(['ip', 'os', 'device']).click_time.shift(+1)).astype(np.float32,copy=False)

# feature3:
# print(train_df[['ip','app']].groupby(['ip','app']).size())
# print(train_df[['ip','app', 'channel']].groupby(['ip', 'app'])[['channel']].count()) # 完全相同
# print(train_df[['ip','app']].groupby(['ip','app']).count()) # 必须任意加一个无关的列
# train_df = do_countuniq( train_df, ['ip'], 'channel', 'B0', show_max=False)
# train_df['nextClick6'] = (train_df.groupby(['ip', 'os', 'channel','app']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)
# train_df = do_count( train_df, ['app', 'channel'], 'B1', show_max=False )
# train_df = do_count( train_df, ['app', 'channel','hour'], 'B2', show_max=False )
# train_df = do_count( train_df, ['app', 'os'], 'B3', show_max=False )
# train_df = do_count( train_df, ['app', 'os','channel'], 'B4', show_max=False )

# feature4:
train_df = do_mean( train_df, ['ip', 'app'], 'channel', 'C0', show_max=False )
train_df = do_mean( train_df, ['ip', 'device'], 'channel', 'C1', show_max=False )


# train_df.drop(['click_time','ip','day','channel','is_attributed','os','device','app','click_id','hour'], axis=1, inplace=True)
train_df.drop(['ip','channel','device','app'], axis=1, inplace=True)
gc.collect()

#----------------------------------------------------------------------------------------------------------------
print("vars and data type: ")
# Warning:A value is trying to be set on a copy of a slice from a DataFrame: only test_df
test_df = train_df[len_train:]
val_df = train_df[(len_train-val_size):len_train] # Validation set
train_df = train_df[:(len_train-val_size)]
gc.collect()

print('Generate real testset from test_supplement...')
test_df_real = test_df[21290876:27493809]
test_df_real = pd.concat([test_df_real,test_df[35678696:41791910]], copy=False)
test_df_real = pd.concat([test_df_real,test_df[48109937:54584259]], copy=False)
test_df = test_df_real
gc.collect()

print("train size: ", len(train_df),train_df.head())
print("valid size: ", len(val_df))
print("test size : ", len(test_df))
print(test_df.columns)

print('Converting train/valid into float32...')
train_df = train_df.astype('float32',copy=False) # categories类型也转换为float32：精度没有问题
val_df = val_df.astype('float32',copy=False)
print(train_df.info())
print(val_df.info())
print(test_df.info())
gc.collect()

print('Saving data to disk...')
train_df.to_pickle('./train_df_feature4.pkl')
val_df.to_pickle('./val_df_feature4.pkl')
test_df.to_pickle('./test_df_feature4.pkl')

loading train data... 0 to 184903891
loading test data...
Combine test_df and train_df...
Extracting hour and day and add 8h...
Combining new features...
Calculating mean of  channel  by  ['ip', 'app'] ...
Calculating mean of  channel  by  ['ip', 'device'] ...
vars and data type: 
Generate real testset from test_supplement...
train size:  122070801            C0          C1
0  278.586273  264.425903
1  279.262939  262.869843
2  271.784485  254.209656
3  397.849213  274.730377
4  294.650848  278.970825
valid size:  62833089
test size :  18790469
Index(['C0', 'C1'], dtype='object')
Converting train/valid into float32...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 122070801 entries, 0 to 122070800
Data columns (total 2 columns):
C0    float32
C1    float32
dtypes: float32(2)
memory usage: 1.8 GB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 62833089 entries, 122070801 to 184903889
Data columns (total 2 columns):
C0    float32
C1    float32
dtypes: float32(2)
memory usage: 9